In [1]:
%%writefile app.py
# ============================================================
# app.py — Guest Experience Intelligence for Restaurants
# Premium UI + internal ML/DL models (hidden from end user)
# ============================================================

import re
import joblib
import torch
import pandas as pd
import streamlit as st

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# ------------------------------------------------------------
# NLTK setup (used for simple text cleaning)
# ------------------------------------------------------------
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# ------------------------------------------------------------
# Streamlit page configuration + luxury dark theme CSS
# ------------------------------------------------------------
st.set_page_config(
    page_title="Guest Experience Intelligence",
    page_icon=None,
    layout="wide",
)

LUXURY_CSS = """
<style>
/* Global */
body {
    background-color: #050608;
    color: #f5f5f5;
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
}

section.main > div {
    padding-top: 10px;
}

/* Titles */
.main-title {
    font-size: 34px;
    font-weight: 600;
    letter-spacing: 0.03em;
    color: #f7f7f7;
    margin-bottom: 4px;
}

.main-subtitle {
    font-size: 14px;
    color: #b7b7b7;
    margin-bottom: 20px;
}

/* Accent line under title */
.accent-line {
    width: 120px;
    height: 2px;
    background: linear-gradient(90deg, #d4af37, #b48b22);
    margin: 6px 0 18px 0;
}

/* Cards / result boxes */
.result-box, .metric-card {
    background-color: #111318;
    padding: 18px 20px;
    border-radius: 12px;
    border: 1px solid #262833;
    box-shadow: 0 14px 40px rgba(0,0,0,0.40);
}

/* Section labels */
.section-label {
    font-size: 12px;
    text-transform: uppercase;
    letter-spacing: 0.16em;
    color: #a1a1aa;
    margin-bottom: 6px;
}

/* Tabs */
.stTabs [data-baseweb="tab"] {
    font-size: 14px;
    font-weight: 500;
    color: #d4d4d8;
}
.stTabs [data-baseweb="tab"][aria-selected="true"] {
    color: #fefce8;
    border-bottom: 2px solid #eab308;
}

/* Text area */
.stTextArea textarea {
    background-color: #050608;
    color: #f9fafb;
    border-radius: 10px;
    border: 1px solid #27272f;
    font-size: 14px;
}

/* Buttons */
.stButton>button {
    background: linear-gradient(90deg, #d4af37, #b48b22);
    color: #050608;
    border-radius: 999px;
    padding: 6px 26px;
    font-size: 14px;
    font-weight: 500;
    border: none;
}
.stButton>button:hover {
    background: linear-gradient(90deg, #eab308, #d97706);
    color: #050608;
}

/* Dataframe */
.dataframe td, .dataframe th {
    color: #e5e7eb !important;
}

/* Footer */
.footer-text {
    font-size: 11px;
    color: #71717a;
    margin-top: 28px;
}
</style>
"""
st.markdown(LUXURY_CSS, unsafe_allow_html=True)

# ------------------------------------------------------------
# Load ALL MODELS (hidden from restaurant user)
# ------------------------------------------------------------
@st.cache_resource(show_spinner=True)
def load_models():
    """
    Loads the traditional model, the deep learning model and encoders.
    This runs only once per session and is hidden from the restaurant user.
    """
    # Traditional text model (used internally)
    tfidf_vectorizer = joblib.load("models/tfidf_vectorizer_lr.pkl")
    lr_model = joblib.load("models/logistic_regression_model.pkl")

    # Transformer model for richer understanding of text
    tokenizer = AutoTokenizer.from_pretrained("models/distilbert_pytorch_model")
    bert_model = AutoModelForSequenceClassification.from_pretrained(
        "models/distilbert_pytorch_model"
    )

    # Best available device
    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    bert_model.to(device)

    label_encoder = joblib.load("models/distilbert_label_encoder.pkl")

    return tfidf_vectorizer, lr_model, tokenizer, bert_model, label_encoder, device


tfidf, lr_model, tokenizer_bert, bert_model, label_encoder_bert, device = load_models()

# ------------------------------------------------------------
# Aspect + Emotion Dictionaries
# ------------------------------------------------------------
aspect_keywords = {
    "Food": ["food", "meal", "dish", "taste", "flavour", "flavor"],
    "Service": ["service", "staff", "waiter", "server", "host", "hostess"],
    "Ambience": ["ambience", "ambiance", "atmosphere", "music", "lighting", "decor"],
    "Pricing": ["price", "expensive", "cheap", "cost", "value"],
    "Location": ["location", "place", "area", "neighbourhood", "neighborhood"],
}

emotion_lexicon = {
    "Joy": ["good", "love", "excellent", "happy", "great", "tasty", "delicious"],
    "Frustration": ["rude", "slow", "annoying", "ignored", "waited"],
    "Disappointment": ["disappointed", "sad", "upset", "unhappy", "letdown"],
    "Disgust": ["dirty", "disgusting", "stale", "smelled", "smelly"],
    "Surprise": ["unexpected", "surprising", "impressed", "shocked"],
    "Concern": ["unsafe", "scary", "worried"],
}

# ------------------------------------------------------------
# Text Cleaning + Helpers
# ------------------------------------------------------------
def clean_text(text: str) -> str:
    """Simple text cleaning used before passing text to the models."""
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    words = [
        lemmatizer.lemmatize(w)
        for w in text.split()
        if w not in stop_words
    ]
    return " ".join(words)


def extract_aspects(cleaned_text: str):
    """Keyword-based aspect detection."""
    found = []
    for aspect, keys in aspect_keywords.items():
        for k in keys:
            if k in cleaned_text:
                found.append(aspect)
                break
    return found if found else ["General"]


def detect_emotion(cleaned_text: str) -> str:
    """Very light-weight keyword-based emotion estimate."""
    for emotion, keys in emotion_lexicon.items():
        for k in keys:
            if k in cleaned_text:
                return emotion
    return "Neutral"


# ------------------------------------------------------------
# Prediction Functions (internal)
# ------------------------------------------------------------
def predict_traditional(text: str) -> str:
    """Uses the traditional model internally (not shown to the owner)."""
    cleaned = clean_text(text)
    vec = tfidf.transform([cleaned])
    pred = lr_model.predict(vec)[0]
    return str(pred)


def predict_transformer(text: str) -> str:
    """Uses the transformer model to estimate overall sentiment."""
    encoded = tokenizer_bert(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}

    with torch.no_grad():
        outputs = bert_model(**encoded)

    label_id = torch.argmax(outputs.logits, dim=1).item()
    sentiment_label = label_encoder_bert.inverse_transform([label_id])[0]
    return str(sentiment_label)


def summarise_sentiment_label(raw_label: str) -> str:
    """Maps raw labels to neat phrases for restaurant owners."""
    raw = raw_label.lower()
    if raw == "positive":
        return "Overall positive"
    if raw == "negative":
        return "Mainly negative"
    if raw == "neutral":
        return "Mixed / neutral"
    return raw_label.capitalize()


def generate_action(aspects, sentiment: str) -> str:
    """Business-friendly recommendation text based on aspects + sentiment."""
    if not isinstance(aspects, list):
        aspects = [aspects]

    sentiment = sentiment.lower()

    if sentiment == "negative":
        if "Service" in aspects:
            return "Guests are unhappy with service. Focus on response times, courtesy and attention at the table."
        if "Food" in aspects:
            return "Guests are not satisfied with food. Reviewing taste consistency, portion size and presentation may help."
        if "Ambience" in aspects:
            return "Atmosphere is mentioned negatively. Lighting, music volume and cleanliness may need attention."
        if "Pricing" in aspects:
            return "Pricing feels sensitive. Consider checking value for money or reviewing special offers."
        return "Several guests are dissatisfied. Reviewing recent operations and feedback in detail is recommended."

    if sentiment == "positive":
        return "Guests are generally happy. Keeping current standards and capturing reviews as testimonials is advised."

    # neutral / mixed
    if "Service" in aspects and "Food" in aspects:
        return "Feedback is mixed on both food and service. A closer look at peak hours and staff workload may reveal issues."
    return "Feedback is mixed. Reviewing comments for each topic can show where small changes will have the biggest impact."


# ============================================================
# USER INTERFACE (what the restaurant owner sees)
# ============================================================

st.markdown('<div class="main-title">Guest Experience Intelligence</div>', unsafe_allow_html=True)
st.markdown('<div class="accent-line"></div>', unsafe_allow_html=True)
st.markdown(
    '<div class="main-subtitle">A focused dashboard for understanding guest feedback and prioritising improvements.</div>',
    unsafe_allow_html=True,
)

tab_single, tab_bulk = st.tabs(["Single Guest Comment", "Review Dashboard"])

# ------------------------------------------------------------
# TAB 1 — Single Guest Comment
# ------------------------------------------------------------
with tab_single:
    st.markdown('<div class="section-label">Guest comment</div>', unsafe_allow_html=True)
    review_text = st.text_area(
        "Paste a recent guest comment below.",
        height=130,
        label_visibility="collapsed",
    )

    if st.button("Analyse Comment", key="analyse_single"):
        if review_text.strip() == "":
            st.warning("Please paste a guest comment first.")
        else:
            cleaned = clean_text(review_text)
            aspects = extract_aspects(cleaned)
            emotion = detect_emotion(cleaned)

            sentiment_main = predict_transformer(review_text)
            sentiment_phrase = summarise_sentiment_label(sentiment_main)

            # Traditional model is used internally; not shown to the owner
            _ = predict_traditional(review_text)

            st.markdown('<div class="result-box">', unsafe_allow_html=True)
            st.markdown('<div class="section-label">Overview</div>', unsafe_allow_html=True)
            st.write("The system has read the comment and summarised how the guest feels.")

            col_a, col_b, col_c = st.columns(3)

            with col_a:
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.markdown("**Overall mood**")
                st.markdown(f"{sentiment_phrase}")
                st.markdown('</div>', unsafe_allow_html=True)

            with col_b:
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.markdown("**Main topics**")
                st.markdown(", ".join(aspects))
                st.markdown('</div>', unsafe_allow_html=True)

            with col_c:
                st.markdown('<div class="metric-card">', unsafe_allow_html=True)
                st.markdown("**Guest feeling**")
                st.markdown(emotion)
                st.markdown('</div>', unsafe_allow_html=True)

            st.markdown("<br/>", unsafe_allow_html=True)
            st.markdown("**Suggested follow-up**")
            st.info(generate_action(aspects, sentiment_main))

            st.markdown('</div>', unsafe_allow_html=True)

# ------------------------------------------------------------
# TAB 2 — Bulk Review Dashboard
# ------------------------------------------------------------
with tab_bulk:
    st.markdown('<div class="section-label">Review file</div>', unsafe_allow_html=True)
    uploaded_file = st.file_uploader(
        "Upload a CSV file that contains a column called 'Review'.",
        type=["csv"],
    )

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
        except Exception as e:
            st.error(f"Could not read file. Details: {e}")
            df = None

        if df is not None:
            if "Review" not in df.columns:
                st.error("The file must contain a column named 'Review'.")
            else:
                st.success("File loaded successfully. Analysing all reviews…")

                # Apply pipeline
                df["Cleaned"] = df["Review"].astype(str).apply(clean_text)
                df["Topics"] = df["Cleaned"].apply(extract_aspects)
                df["Guest_Feeling"] = df["Cleaned"].apply(detect_emotion)
                df["Overall_Sentiment"] = df["Review"].astype(str).apply(predict_transformer)

                # A small sample table
                st.markdown('<div class="result-box">', unsafe_allow_html=True)
                st.markdown('<div class="section-label">Sample of analysed reviews</div>', unsafe_allow_html=True)
                st.dataframe(
                    df[["Review", "Overall_Sentiment", "Guest_Feeling", "Topics"]].head()
                )
                st.markdown('</div>', unsafe_allow_html=True)

                # Sentiment distribution
                st.markdown('<div class="section-label">Sentiment overview</div>', unsafe_allow_html=True)
                sentiment_order = ["negative", "neutral", "positive"]
                sentiment_counts = (
                    df["Overall_Sentiment"]
                    .str.lower()
                    .value_counts()
                    .reindex(sentiment_order, fill_value=0)
                )
                st.bar_chart(sentiment_counts)

                # Download full analysed file
                csv_bytes = df.to_csv(index=False).encode("utf-8")
                st.download_button(
                    "Download full analysis as CSV",
                    data=csv_bytes,
                    file_name="guest_feedback_analysis.csv",
                    mime="text/csv",
                )

# ------------------------------------------------------------
# Footer
# ------------------------------------------------------------
st.markdown(
    '<div class="footer-text">Dashboard generated for research purposes. '
    'Designed to support restaurant teams in understanding guest feedback.</div>',
    unsafe_allow_html=True,
)


Overwriting app.py


In [5]:
%%writefile app.py
# ============================================================
# app.py — Guest Experience Intelligence for Restaurants
# Premium UI + internal ML/DL models
# ============================================================

import re
import joblib
import torch
import pandas as pd
import streamlit as st

from transformers import AutoTokenizer, AutoModelForSequenceClassification

import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# ------------------------------------------------------------
# NLTK setup
# ------------------------------------------------------------
nltk.download("stopwords", quiet=True)
nltk.download("wordnet", quiet=True)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

# ------------------------------------------------------------
# Streamlit Page Setup + Clean Gold-Black Theme
# ------------------------------------------------------------
st.set_page_config(
    page_title="Guest Experience Intelligence",
    page_icon=None,
    layout="wide",
    initial_sidebar_state="collapsed",
)

CLEAN_GOLD_THEME = """
<style>
/* MAIN BACKGROUND */
.stApp {
    background-color: #050608;
    color: #f5f5f5;
    font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", sans-serif;
}

/* MAIN CONTAINER */
main.block-container {
    max-width: 1200px;
    padding-top: 1rem;
}

/* TITLES */
.main-title {
    font-size: 34px;
    font-weight: 650;
    color: #ffffff;
    letter-spacing: 0.02em;
}
.main-subtitle {
    font-size: 15px;
    color: #c9c9c9;
    margin-bottom: 18px;
}
.accent-line {
    width: 130px;
    height: 2px;
    background: linear-gradient(90deg, #d4af37, #facc15);
    margin: 5px 0 20px 0;
}

/* SECTION LABELS */
.section-label {
    font-size: 11px;
    text-transform: uppercase;
    letter-spacing: 0.20em;
    color: #a1a1aa;
    margin-bottom: 6px;
}

/* CLEAN BLACK CARDS WITH GOLD BORDER */
.result-box, .metric-card {
    background-color: #0a0a0b;
    border-radius: 12px;
    border: 1px solid #d4af37;
    padding: 16px 18px;
    color: white !important;
}

/* FIX TEXT VISIBILITY INSIDE CARDS */
.result-box *, .metric-card * {
    color: #ffffff !important;
}

/* TABS */
.stTabs [data-baseweb="tab"] {
    font-size: 14px;
    font-weight: 500;
    color: #d1d1d1;
}
.stTabs [data-baseweb="tab"][aria-selected="true"] {
    color: #fefce8;
    border-bottom: 2px solid #eab308;
}

/* TEXT AREA */
.stTextArea textarea {
    background-color: #0d0d0f;
    border: 1px solid #d4af37;
    border-radius: 12px;
    color: #ffffff !important;
    padding: 14px;
    font-size: 14px;
}

/* BUTTONS */
.stButton>button {
    background: linear-gradient(90deg, #facc15, #d4af37);
    color: #000000;
    border-radius: 999px;
    padding: 8px 30px;
    font-size: 14px;
    font-weight: 600;
    border: none;
}
.stButton>button:hover {
    background: linear-gradient(90deg, #fde047, #facc15);
    color: #000000;
}

/* DATAFRAME WRAPPER */
div[data-testid="stDataFrame"] {
    border-radius: 12px;
    border: 1px solid #d4af37;
    background-color: #0d0d0f;
}

/* FOOTER */
.footer-text {
    font-size: 11px;
    color: #9ca3af;
    margin-top: 30px;
}
</style>
"""
st.markdown(CLEAN_GOLD_THEME, unsafe_allow_html=True)

# ------------------------------------------------------------
# Load Models
# ------------------------------------------------------------
@st.cache_resource(show_spinner=True)
def load_models():
    tfidf_vectorizer = joblib.load("models/tfidf_vectorizer_lr.pkl")
    lr_model = joblib.load("models/logistic_regression_model.pkl")

    tokenizer = AutoTokenizer.from_pretrained("models/distilbert_pytorch_model")
    bert_model = AutoModelForSequenceClassification.from_pretrained(
        "models/distilbert_pytorch_model"
    )

    if torch.backends.mps.is_available():
        device = torch.device("mps")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")

    bert_model.to(device)
    label_encoder = joblib.load("models/distilbert_label_encoder.pkl")

    return tfidf_vectorizer, lr_model, tokenizer, bert_model, label_encoder, device


tfidf, lr_model, tokenizer_bert, bert_model, label_encoder_bert, device = load_models()

# ------------------------------------------------------------
# Dictionaries
# ------------------------------------------------------------
aspect_keywords = {
    "Food": ["food", "meal", "dish", "taste", "flavour", "flavor"],
    "Service": ["service", "staff", "waiter", "server", "host", "hostess"],
    "Ambience": ["ambience", "ambiance", "atmosphere", "music", "lighting", "decor"],
    "Pricing": ["price", "expensive", "cheap", "cost", "value"],
    "Location": ["location", "place", "area", "neighbourhood", "neighborhood"],
}

emotion_lexicon = {
    "Joy": ["good", "love", "excellent", "happy", "great", "tasty", "delicious"],
    "Frustration": ["rude", "slow", "annoying", "ignored", "waited"],
    "Disappointment": ["disappointed", "sad", "upset", "unhappy", "letdown"],
    "Disgust": ["dirty", "disgusting", "stale", "smelled", "smelly"],
    "Surprise": ["unexpected", "surprising", "impressed", "shocked"],
    "Concern": ["unsafe", "scary", "worried"],
}

# ------------------------------------------------------------
# Helpers
# ------------------------------------------------------------
def clean_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    text = text.lower()
    text = re.sub(r"[^\w\s]", " ", text)
    words = [
        lemmatizer.lemmatize(w)
        for w in text.split()
        if w not in stop_words
    ]
    return " ".join(words)


def extract_aspects(cleaned_text: str):
    found = []
    for aspect, keys in aspect_keywords.items():
        for k in keys:
            if k in cleaned_text:
                found.append(aspect)
                break
    return found if found else ["General"]


def detect_emotion(cleaned_text: str):
    for emotion, keys in emotion_lexicon.items():
        for k in keys:
            if k in cleaned_text:
                return emotion
    return "Neutral"


def predict_traditional(text: str):
    cleaned = clean_text(text)
    vec = tfidf.transform([cleaned])
    pred = lr_model.predict(vec)[0]
    return str(pred)


def predict_transformer(text: str):
    encoded = tokenizer_bert(
        text,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=128,
    )
    encoded = {k: v.to(device) for k, v in encoded.items()}

    with torch.no_grad():
        outputs = bert_model(**encoded)

    label_id = torch.argmax(outputs.logits, dim=1).item()
    return label_encoder_bert.inverse_transform([label_id])[0]


def summarise_sentiment_label(raw_label: str) -> str:
    raw = raw_label.lower()
    if raw == "positive":
        return "Overall positive"
    if raw == "negative":
        return "Mainly negative"
    if raw == "neutral":
        return "Mixed / neutral"
    return raw_label.capitalize()


def generate_action(aspects, sentiment: str) -> str:
    if not isinstance(aspects, list):
        aspects = [aspects]

    sentiment = sentiment.lower()

    if sentiment == "negative":
        if "Service" in aspects:
            return "Guests are unhappy with service. Improve response times and courtesy."
        if "Food" in aspects:
            return "Guests are not satisfied with food. Review taste consistency and presentation."
        if "Ambience" in aspects:
            return "Atmosphere concerns raised. Check lighting and cleanliness."
        if "Pricing" in aspects:
            return "Pricing seems sensitive. Review value for money."
        return "Guests show dissatisfaction. Review feedback in detail."

    if sentiment == "positive":
        return "Guests are happy. Continue delivering good experiences."

    if "Service" in aspects and "Food" in aspects:
        return "Mixed feedback on food and service. Check peak-hour workload."
    return "Mixed feedback. Small adjustments may help."


# ------------------------------------------------------------
# UI
# ------------------------------------------------------------
st.markdown('<div class="main-title">Guest Experience Intelligence</div>', unsafe_allow_html=True)
st.markdown('<div class="accent-line"></div>', unsafe_allow_html=True)
st.markdown(
    '<div class="main-subtitle">A focused dashboard for understanding guest feedback and prioritising improvements.</div>',
    unsafe_allow_html=True,
)

tab_single, tab_bulk = st.tabs(["Single Guest Comment", "Review Dashboard"])

# ------------------------------------------------------------
# TAB 1 — SINGLE COMMENT
# ------------------------------------------------------------
with tab_single:
    st.markdown('<div class="section-label">Guest comment</div>', unsafe_allow_html=True)

    review_text = st.text_area(
        "Paste a recent guest comment below.",
        height=140,
        label_visibility="collapsed",
    )

    if st.button("Analyse Comment"):
        if review_text.strip() == "":
            st.warning("Please paste a guest comment first.")
        else:
            cleaned = clean_text(review_text)
            aspects = extract_aspects(cleaned)
            emotion = detect_emotion(cleaned)
            sentiment_main = predict_transformer(review_text)
            sentiment_phrase = summarise_sentiment_label(sentiment_main)

            st.markdown('<div class="result-box">', unsafe_allow_html=True)
            st.markdown('<div class="section-label">Overview</div>', unsafe_allow_html=True)
            st.write("The system has read the comment and summarised how the guest feels.")

            col1, col2, col3 = st.columns(3)

            with col1:
                st.markdown('<div class="metric-card">Overall mood<br><b>' + sentiment_phrase + '</b></div>', unsafe_allow_html=True)

            with col2:
                st.markdown('<div class="metric-card">Main topics<br><b>' + ", ".join(aspects) + '</b></div>', unsafe_allow_html=True)

            with col3:
                st.markdown('<div class="metric-card">Guest feeling<br><b>' + emotion + '</b></div>', unsafe_allow_html=True)

            st.markdown("**Suggested follow-up**")
            st.info(generate_action(aspects, sentiment_main))

            st.markdown('</div>', unsafe_allow_html=True)

# ------------------------------------------------------------
# TAB 2 — BULK REVIEWS
# ------------------------------------------------------------
with tab_bulk:
    st.markdown('<div class="section-label">Review file</div>', unsafe_allow_html=True)

    uploaded_file = st.file_uploader(
        "Upload a CSV file with a column named 'Review'.",
        type=["csv"],
    )

    if uploaded_file is not None:
        try:
            df = pd.read_csv(uploaded_file)
        except Exception as e:
            st.error(f"Could not read file: {e}")
            df = None

        if df is not None:
            if "Review" not in df.columns:
                st.error("The file must contain a column named 'Review'.")
            else:
                st.success("File loaded successfully. Analysing…")

                df["Cleaned"] = df["Review"].astype(str).apply(clean_text)
                df["Topics"] = df["Cleaned"].apply(extract_aspects)
                df["Guest_Feeling"] = df["Cleaned"].apply(detect_emotion)
                df["Overall_Sentiment"] = df["Review"].astype(str).apply(predict_transformer)

                st.markdown('<div class="result-box">', unsafe_allow_html=True)
                st.markdown('<div class="section-label">Sample results</div>', unsafe_allow_html=True)

                st.dataframe(df[["Review", "Overall_Sentiment", "Guest_Feeling", "Topics"]].head())

                st.markdown('</div>', unsafe_allow_html=True)

                st.markdown('<div class="section-label">Sentiment overview</div>', unsafe_allow_html=True)

                sentiment_order = ["negative", "neutral", "positive"]
                sentiment_counts = (
                    df["Overall_Sentiment"].str.lower().value_counts().reindex(sentiment_order, fill_value=0)
                )
                st.bar_chart(sentiment_counts)

                # DOWNLOAD BUTTON
                csv_bytes = df.to_csv(index=False).encode("utf-8")
                st.download_button(
                    "Download CSV results",
                    data=csv_bytes,
                    file_name="guest_feedback_analysis.csv",
                    mime="text/csv",
                )

# ------------------------------------------------------------
# FOOTER
# ------------------------------------------------------------
st.markdown(
    '<div class="footer-text">Dashboard generated for research purposes. Designed to support restaurant teams.</div>',
    unsafe_allow_html=True,
)


Overwriting app.py
